In [1]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers, optimizers
import numpy as np


(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

Using TensorFlow backend.


In [2]:
# Swish activation function
# x*sigmoid(x)
def swish(x):
    return x*K.sigmoid(x)

# Custom activation function 1
# mix between relu and positive part of swish mirrored across x=1
def e_swish_1(x):
    return K.maximum(0.0, x*(2-K.sigmoid(x)))

# Custom activation function 2
# positive part of swish mirrored across x=1
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [3]:
activations = [e_swish_1, e_swish_2, swish, "relu", "sigmoid", "tanh"]
names = activations[:]
for i,a in enumerate(names):
    if not isinstance(a, str):
        names[i] = a.__name__
print(names)

['e_swish_1', 'e_swish_2', 'swish', 'relu', 'sigmoid', 'tanh']


In [4]:
baseMapNum = 32
weight_decay = 1e-4

act, act_name = activations[2], names[2]
print(act_name)


model = Sequential()
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(2*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(Conv2D(4*baseMapNum, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation(act))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

model.summary()

swish
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
____

In [5]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [6]:
#training
batch_size = 128
epochs=25
opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=3*epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep75'+act_name+'.h5')

Epoch 1/75
390/390 [==============================] - 155s - loss: 1.9323 - acc: 0.4188 - val_loss: 1.7221 - val_acc: 0.4713
Epoch 2/75
390/390 [==============================] - 152s - loss: 1.3531 - acc: 0.5720 - val_loss: 1.0408 - val_acc: 0.6731
Epoch 3/75
390/390 [==============================] - 151s - loss: 1.2038 - acc: 0.6329 - val_loss: 1.0127 - val_acc: 0.7021
Epoch 4/75
390/390 [==============================] - 151s - loss: 1.1117 - acc: 0.6674 - val_loss: 0.8938 - val_acc: 0.7361
Epoch 5/75
390/390 [==============================] - 150s - loss: 1.0206 - acc: 0.6943 - val_loss: 0.8497 - val_acc: 0.7431
Epoch 6/75
390/390 [==============================] - 150s - loss: 1.0421 - acc: 0.7026 - val_loss: 0.7800 - val_acc: 0.7689
Epoch 7/75
390/390 [==============================] - 150s - loss: 0.9491 - acc: 0.7197 - val_loss: 0.7157 - val_acc: 0.7880
Epoch 8/75
390/390 [==============================] - 150s - loss: 0.9149 - acc: 0.7342 - val_loss: 0.6919 - val_acc: 0.7979


390/390 [==============================] - 151s - loss: 0.6695 - acc: 0.8327 - val_loss: 0.5424 - val_acc: 0.8671
Epoch 67/75
390/390 [==============================] - 151s - loss: 0.6550 - acc: 0.8350 - val_loss: 0.5180 - val_acc: 0.8712
Epoch 68/75
390/390 [==============================] - 150s - loss: 0.6588 - acc: 0.8339 - val_loss: 0.5597 - val_acc: 0.8631
Epoch 69/75
390/390 [==============================] - 150s - loss: 0.6581 - acc: 0.8379 - val_loss: 0.5244 - val_acc: 0.8684
Epoch 70/75
390/390 [==============================] - 150s - loss: 0.6521 - acc: 0.8370 - val_loss: 0.5658 - val_acc: 0.8605
Epoch 71/75
390/390 [==============================] - 150s - loss: 0.6470 - acc: 0.8365 - val_loss: 0.5517 - val_acc: 0.8634
Epoch 72/75
390/390 [==============================] - 150s - loss: 0.6532 - acc: 0.8369 - val_loss: 0.5375 - val_acc: 0.8651
Epoch 73/75
390/390 [==============================] - 150s - loss: 0.6559 - acc: 0.8363 - val_loss: 0.5371 - val_acc: 0.8720
Epoc

In [7]:
opt_rms = keras.optimizers.rmsprop(lr=0.0005,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep100'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 151s - loss: 0.6043 - acc: 0.8498 - val_loss: 0.4878 - val_acc: 0.8841
Epoch 2/25
390/390 [==============================] - 150s - loss: 0.5879 - acc: 0.8557 - val_loss: 0.4923 - val_acc: 0.8858
Epoch 3/25
390/390 [==============================] - 150s - loss: 0.5832 - acc: 0.8567 - val_loss: 0.4915 - val_acc: 0.8853
Epoch 4/25
390/390 [==============================] - 150s - loss: 0.5801 - acc: 0.8596 - val_loss: 0.4907 - val_acc: 0.8898
Epoch 5/25
390/390 [==============================] - 150s - loss: 0.5700 - acc: 0.8584 - val_loss: 0.5023 - val_acc: 0.8825
Epoch 6/25
390/390 [==============================] - 150s - loss: 0.5676 - acc: 0.8604 - val_loss: 0.4728 - val_acc: 0.8867
Epoch 7/25
390/390 [==============================] - 150s - loss: 0.5685 - acc: 0.8608 - val_loss: 0.4883 - val_acc: 0.8866
Epoch 8/25
390/390 [==============================] - 150s - loss: 0.5606 - acc: 0.8600 - val_loss: 0.4678 - val_acc: 0.8853


In [8]:
opt_rms = keras.optimizers.rmsprop(lr=0.0003,decay=1e-6)
model.compile(loss='categorical_crossentropy',
        optimizer=opt_rms,
        metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test))
model.save_weights('cifar10_normal_rms_ep125'+act_name+'.h5')

Epoch 1/25
390/390 [==============================] - 151s - loss: 0.5076 - acc: 0.8729 - val_loss: 0.4521 - val_acc: 0.8930
Epoch 2/25
390/390 [==============================] - 150s - loss: 0.5077 - acc: 0.8730 - val_loss: 0.4312 - val_acc: 0.8975
Epoch 3/25
390/390 [==============================] - 150s - loss: 0.5029 - acc: 0.8758 - val_loss: 0.4359 - val_acc: 0.8955
Epoch 4/25
390/390 [==============================] - 150s - loss: 0.4987 - acc: 0.8739 - val_loss: 0.4276 - val_acc: 0.8954
Epoch 5/25
390/390 [==============================] - 150s - loss: 0.4902 - acc: 0.8760 - val_loss: 0.4383 - val_acc: 0.8934
Epoch 6/25
390/390 [==============================] - 150s - loss: 0.4975 - acc: 0.8759 - val_loss: 0.4396 - val_acc: 0.8930
Epoch 7/25
390/390 [==============================] - 150s - loss: 0.4911 - acc: 0.8757 - val_loss: 0.4271 - val_acc: 0.8966
Epoch 8/25
390/390 [==============================] - 150s - loss: 0.4867 - acc: 0.8784 - val_loss: 0.4479 - val_acc: 0.8920


In [9]:
#testing - no kaggle eval
scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

 9984/10000 [============================>.] - ETA: 0s
Test result: 89.700 loss: 0.417
